In [ ]:
%pip install pandas
%pip install transformers
%pip install accelerate
%pip install xformers
%pip install spacy

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


### Refer the github repository 'https://github.com/unslothai/unsloth' for more information about fine-tuning with unsloth

In [ ]:
%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit" ,        # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-it",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/ltstorage/home/1samreen/miniconda3/envs/new_saint_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.138 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Data creation and processing

In [5]:
import pandas as pd
df = pd.read_csv('train_subset.csv')
pd.options.display.max_colwidth = 1000
df

,user,instance_id,text,sentiment,negative object_spans,negative spans,positive spans,positive object_spans
0,ekaterini@gmail.com,1237,die pandemie mag zu ende sein. viele geimpfte stecken weiter in ihren impfschäden. 🙁,negative,die pandemie,"['viele geimpfte stecken weiter in ihren impfschäden', '🙁']",...,...
1,ekaterini@gmail.com,210,"Wir sind hier nicht bei 'Wünsch dir was', sondern bei 'So isses' und das gilt auch für den schlechtesten Bundesminister für #Wirtschaft und #Klimaschutz Robert #Habeck \nKeinen Bock mehr...was ist denn das für eine Aussage?",negative,"['Klimaschutz', 'Habeck ']","['schlechtesten Bundesminister', 'Keinen Bock mehr']",...,...
2,ekaterini@gmail.com,494,"Es kann wirklich niemand mehr behaupten, dass man nicht wusste, was man wählt!",neutral,...,...,...,...
3,ekaterini@gmail.com,123,Da ist der falsche bestraft wurden,negative,...,der falsche bestraft wurden,...,...
4,ekaterini@gmail.com,1470,"Vor 500 Jahren hätte man gesagt, die #AfD sei des Teufels Wiedergänger. Man hätte sie geteert und gefedert, auf einen Holzpfahl gespießt, mit Steinen beschwert und fünf Meter tief vergraben. Nun leben wir in einer Demokratie und müssen sie ertragen. Aber #LautgegenRechts dürfen wir sein!",negative,#AfD,"['Man hätte sie geteert und gefedert, auf einen Holzpfahl gespießt, mit Steinen beschwert und fünf Meter tief vergraben.', 'Nun leben wir in einer Demokratie und müssen sie ertragen']",...,...
...,...,...,...,...,...,...,...,...
1045,ekaterini@gmail.com,1443,Zwei wahre Sätze für den Abend:\nJedes einzelne zivile Opfer im Gazastreifen geht auf das Konto der Hamas. Keine relevante Armee der Welt handelt nach höheren moralischen Maßstäben als die IDF.,negative,Hamas,zivile Opfer im Gazastreifen,...,...
1046,ekaterini@gmail.com,4,Danke Südafrika👏 wenigstens schaut ihr nicht weg oder seid keine Marionette der USA!,positive,...,...,"['Danke Südafrika👏', 'seid keine Marionette']",...
1047,ekaterini@gmail.com,645,"Ohne angeben zu wollen, habe ich #Putin schon vor 15 Jahren absolut alles zugetraut. Der Typ war schon immer total skrupellos & hinterliess nur Schutt & Asche. Nun läuft ihm die Zeit davon, seine Pläne wie gewohnt umzusetzen und die #Demokratie rückt näher an #Russland heran. Der offene Überfall auf die #Ukraine ist klar ein Zeichen dafür. Er wird immer brutaler & unberechenbarer werden. Wir sollten uns darauf einstellen & ihn stoppen, bevor der ganze Kontinent hineingezogen wird.\n\n#NATO #SupportUkraine #Baltikum #Estland #Lettland #Litauen #Georgien #Polen #Moldawien #Finnland\n@Bundeskanzler\n \n@fdp\n \n@ABaerbock\n \n@ignaziocassis\n \n@Die_Gruenen\n \n@spdde\n \n@spdbt",negative,#Putin,"['Der Typ war schon immer total skrupellos', 'hinterliess nur Schutt &amp; Asche.', 'brutaler &amp; unberechenbarer']",...,...
1048,ekaterini@gmail.com,1142,"Bitte Deutschland noch mehr Waffen liefern, bis alle Menschen da getötet werden . Dann kann die Welt friedlich leben oder !? Aber ihr solltet auch nicht vergessen, dass Deutschland und USA sich beteiligt haben , diesen Genozid zu begehen!",negative,...,"['getötet ', 'Aber ihr solltet auch nicht vergessen', 'Genozid ']",...,...


In [6]:
import spacy
nlp = spacy.load('de_core_news_md')

In [7]:
#ignoring neutrals as we do not consider these for NER evaluation
df = df[df['sentiment'] != 'neutral']
df = df.reset_index(drop=True)
len(df)

991

In [8]:
#Creating lists of outputs as a dictionary of 'tokens and labels', instructions and inputs
emojis = ['👏','🤔','🤡','💙','🤝','🥲','🤩','✋','😅','💛','🔥','👍','💔','🤮','💚','🤦','💪','😷','✊','🤢','😭','🙌','😣','🥺','🙁']
inputs = []
outputs = []
instructions = []
#instruction = 'Classify the sentiment of each word in the texts into ONE of the two classes: negative or positive and O if neutral. Split the answer in a dictionary with two keys:Tokens and Labels'
instruction = 'Classify the sentiment of each word in the texts into ONE of the two classes: negative or positive. Split the answer in a dictionary with two keys:Tokens and Labels. Text:'
for i in range(len(df)):
    tokens = []
    labels = []
    inputs.append(df['text'][i])
    instructions.append(instruction)
    tokenized_text=nlp(df['text'][i])
    for token in tokenized_text:
        token = str(token)
        if token in df['negative object_spans'][i] or token in df['negative spans'][i] or token in df['positive object_spans'][i] or token in df['positive spans'][i]:
            if token.isalpha() or token in emojis :
                tokens.append(token)
                labels.append(df['sentiment'][i])
    outputs.append({'Tokens':tokens , 'Labels':labels})
print(len(outputs))


991


In [9]:
df_train_data_LLM = pd.DataFrame(columns=['output','input','instruction'])
df_train_data_LLM['output'] = outputs
df_train_data_LLM['input'] = inputs
df_train_data_LLM['instruction'] = instructions

In [10]:
#Creating a csv file with generated outputs, inputs and instructions to use as training data
df_train_data_LLM.to_csv('train_subset_llm_tokenClassification_latest.csv', index=False)

In [11]:
#Adding the EOS_TOKEN to the tokenized output!! Otherwise you'll get infinite generations!
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train':'train_subset_llm_tokenClassification_latest.csv'})
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 991 examples [00:00, 29820.04 examples/s]
Map: 100%|██████████| 991/991 [00:00<00:00, 55850.41 examples/s]


### Training the model

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 10,
        warmup_steps = 5,
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 500,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "fine-tuning-gemma-withUnsloth-tokenClassification",
    ),
)

In [13]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100 80GB PCIe. Max memory = 79.138 GB.
5.83 GB of memory reserved.


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 991 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,240
 "-____-"     Number of trainable parameters = 50,003,968


Step,Training Loss
500,0.707600
1000,0.069100


### Pushing the model to hugging face hub, for future inference

In [ ]:
model.push_to_hub("<huggingfaceuser>/fine-tuned-gemmaModel_tokenClassification_10epochs") # Online saving
tokenizer.push_to_hub("<huggingfaceuser>/fine-tuned-gemmaModel_tokenClassification_10epochs") # Online saving